In [66]:
import anvil.server

anvil.server.connect("NW5C7J7BP2RMYOHEPOU6LR2U-KAA6QKRCG57NG4NY")

In [67]:
import anvil.media
import pandas as pd

def normalize(df):
    return (df - df.mean()) / df.std()


@anvil.server.callable
def check_fraud(file):
    with anvil.media.TempFile(file) as filename:
        data = pd.read_csv(filename)
    import joblib
    #data = pd.read_csv("upload.csv")
    n = len(data)
    local_data = pd.read_csv("test.csv")
    frames = [local_data, data]
    test_data = pd.concat(frames)
    model = joblib.load(r'Model/model.pickle')
    test_data = test_data[['Card Number', 'Date', 'Amount']]

    card_numbers_to_idx = { v:k for k,v in enumerate(np.unique(test_data['Card Number'].values))}
    test_data['Card Number'].replace(card_numbers_to_idx, inplace=True)
    test_data['Card Number'] = normalize(test_data['Card Number'])

    test_data['Date'] = pd.to_datetime(test_data['Date']).values.astype("datetime64[s]")
    test_data['Date'] = normalize(test_data['Date'])

    test_data['Amount'] = normalize(test_data['Amount'])
    predictions = model.predict(test_data)
    test_data['Fraud'] = predictions
    x = str(test_data['Fraud'].tail(n))
    return(x)